In [1]:
mkdir /home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure4_voyages/

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure4_voyages/': File exists


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from anchor.visualize import MODALITY_TO_COLOR, MODALITY_ORDER, MODALITY_PALETTE
modality_order = MODALITY_ORDER

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline

%load_ext autoreload
%autoreload 2

folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure4_voyages'

import flotilla
study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', 
                        flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')
# study = flotilla.embark('singlecell_pnm_figure1_supplementary_post_splicing_filtering')

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/flotilla/__init__.py:55: DtypeWarning: Columns (11,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


2016-02-22 16:50:50	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2016-02-22 16:50:50	Parsing datapackage to create a Study object
2016-02-22 16:51:28	Initializing Study
2016-02-22 16:51:28	Initializing Predictor configuration manager for Study
2016-02-22 16:51:28	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-02-22 16:51:28	Added ExtraTreesClassifier to default predictors
2016-02-22 16:51:28	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-02-22 16:51:28	Added ExtraTreesRegressor to default predictors
2016-02-22 16:51:28	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-02-22 16:51:28	Added GradientBoostingClassifier to default predictors
2016-02-22 16:51:28	Predictor GradientBoostingRegressor is of type <class 'sklearn.

In [3]:
event = 'exon:chr10:1043148-1043248:+@exon:chr10:1044943-1045035:+@exon:chr10:1046617-1046808:+'

event in study.splicing.data

False

In [4]:
study.supplemental.modalities_tidy.groupby(['phenotype', 'modality']).size()

phenotype  modality  
MN         ambivalent       1
           bimodal        356
           excluded       541
           included      1553
NPC        ambivalent       3
           bimodal        751
           excluded      1049
           included      3029
iPSC       ambivalent      31
           bimodal       1348
           concurrent       1
           excluded      1558
           included      4632
dtype: int64

In [5]:
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=20))

In [6]:
psi_filtered.head()

,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,exon:chr10:101997779-101997988:-@exon:chr10:101996607-101996726:-@exon:chr10:101995424-101995521:-,exon:chr10:102003455-102003534:-@exon:chr10:101997779-101997988:-@exon:chr10:101996607-101996726:-,exon:chr10:102013178-102013296:-@exon:chr10:102010005-102010089:-@exon:chr10:102006552-102006692:-,exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,exon:chr10:102020723-102020801:-@exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-,exon:chr10:102114184-102114389:+@exon:chr10:102116289-102116521:+@exon:chr10:102120491-102124591:+,exon:chr10:102286732-102286831:-@exon:chr10:102286156-102286311:-@exon:chr10:102283500-102283686:-,...,exon:chrY:15024875-15024974:+@exon:chrY:15025630-15025765:+@exon:chrY:15026476-15026561:+,exon:chrY:15026476-15026561:+@exon:chrY:15026796-15026894:+@exon:chrY:15026979-15027139:+,exon:chrY:15027542-15027686:+@exon:chrY:15027795-15027939:+@exon:chrY:15028173-15028354:+,exon:chrY:15028429-15028546:+@exon:chrY:15028819-15028972:+@exon:chrY:15029315-15029454:+,exon:chrY:22744476-22744579:+@exon:chrY:22746360-22746410:+@exon:chrY:22749910-22749991:+,exon:chrY:2709527-2709668:+@exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+,exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+,exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+,exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+,exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2734903:+
CVN_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1
CVN_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1
CVN_05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,NaN,NaN,NaN,NaN,1,1,1,1,1,1


## Calculate *voyage* space via `bon voyage`

In [7]:
from astrolabe import Waypoints

In [8]:
ws = Waypoints()

waypoints = psi_filtered.groupby(study.sample_id_to_phenotype).apply(
    lambda x: ws.fit_transform(x.dropna(how='all', axis=1)))

In [9]:
waypoints.max()

0    1
1    1
dtype: float64

In [10]:
pd.DataFrame(ws.seed_data_transformed).max()

0    0.840892
1    0.999989
dtype: float64

In [11]:
transitions = study.phenotype_transitions + [('iPSC', 'MN')]
transitions

[(u'iPSC', u'NPC'), (u'NPC', u'MN'), ('iPSC', 'MN')]

In [12]:
from astrolabe.voyages import Voyages

In [13]:
v = Voyages()

voyages = v.voyages(waypoints, transitions)
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,exon:chr10:102114184-102114389:+@exon:chr10:10...,0.052632,-0.053237,0.074861,iPSC,NPC,$\searrow$,iPSC-NPC
1,exon:chr10:102286732-102286831:-@exon:chr10:10...,0.000000,0.000000,0.000000,iPSC,NPC,NaN,iPSC-NPC
2,exon:chr10:102747070-102747240:-@exon:chr10:10...,-0.027778,0.027778,0.039284,iPSC,NPC,$\nwarrow$,iPSC-NPC
3,exon:chr10:103348082-103348157:+@exon:chr10:10...,0.000000,-0.028571,0.028571,iPSC,NPC,$\swarrow$,iPSC-NPC
4,exon:chr10:103360960-103361093:+@exon:chr10:10...,-0.056306,0.055556,0.079100,iPSC,NPC,$\nwarrow$,iPSC-NPC


In [14]:
voyages['transition'] = voyages.group1 + '-' + voyages.group2
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,exon:chr10:102114184-102114389:+@exon:chr10:10...,0.052632,-0.053237,0.074861,iPSC,NPC,$\searrow$,iPSC-NPC
1,exon:chr10:102286732-102286831:-@exon:chr10:10...,0.000000,0.000000,0.000000,iPSC,NPC,NaN,iPSC-NPC
2,exon:chr10:102747070-102747240:-@exon:chr10:10...,-0.027778,0.027778,0.039284,iPSC,NPC,$\nwarrow$,iPSC-NPC
3,exon:chr10:103348082-103348157:+@exon:chr10:10...,0.000000,-0.028571,0.028571,iPSC,NPC,$\swarrow$,iPSC-NPC
4,exon:chr10:103360960-103361093:+@exon:chr10:10...,-0.056306,0.055556,0.079100,iPSC,NPC,$\nwarrow$,iPSC-NPC


In [15]:
voyages.shape

(8987, 8)

### Add group1 and group2 modalities

In [18]:
study.supplemental.modalities_tidy.head()

,phenotype,event_id,modality
0,MN,exon:chr10:102114184-102114389:+@exon:chr10:10...,included
1,MN,exon:chr10:102286732-102286831:-@exon:chr10:10...,included
2,MN,exon:chr10:103360960-103361093:+@exon:chr10:10...,bimodal
3,MN,exon:chr10:103563492-103563776:-@exon:chr10:10...,bimodal
4,MN,exon:chr10:103567487-103567658:-@exon:chr10:10...,included


In [20]:
voyages_modalities.head()

,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition,group1_modality
0,0.052632,-0.053237,0.074861,iPSC,NPC,$\searrow$,iPSC-NPC,included
1,0.000000,0.034483,0.034483,iPSC,MN,$\nwarrow$,iPSC-MN,included
2,0.000000,0.000000,0.000000,iPSC,NPC,NaN,iPSC-NPC,included
3,0.000000,0.000000,0.000000,iPSC,MN,NaN,iPSC-MN,included
4,-0.027778,0.027778,0.039284,iPSC,NPC,$\nwarrow$,iPSC-NPC,included


In [22]:
groups = 'group1', 'group2'
voyages_modalities = voyages.copy()

for group in groups:
    voyages_modalities = voyages_modalities.merge(study.supplemental.modalities_tidy, 
                                       left_on=[group, 'event_id'], 
                                       right_on=['phenotype', 'event_id'], copy=False)
    print(voyages_modalities.shape)
    voyages_modalities = voyages_modalities.drop(['phenotype'], axis=1)
    voyages_modalities = voyages_modalities.rename(columns={'modality': '{}_modality'.format(group)})
print(voyages_modalities.shape)
voyages_modalities.head()

(8987, 10)
(8987, 11)
(8987, 10)


,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition,group1_modality,group2_modality
0,exon:chr10:102114184-102114389:+@exon:chr10:10...,0.052632,-0.053237,0.074861,iPSC,NPC,$\searrow$,iPSC-NPC,included,included
1,exon:chr10:102114184-102114389:+@exon:chr10:10...,0.000000,0.034483,0.034483,iPSC,MN,$\nwarrow$,iPSC-MN,included,included
2,exon:chr10:102114184-102114389:+@exon:chr10:10...,-0.052632,0.087719,0.102297,NPC,MN,$\nwarrow$,NPC-MN,included,included
3,exon:chr10:102286732-102286831:-@exon:chr10:10...,0.000000,0.000000,0.000000,iPSC,NPC,NaN,iPSC-NPC,included,included
4,exon:chr10:102286732-102286831:-@exon:chr10:10...,0.000000,0.000000,0.000000,iPSC,MN,NaN,iPSC-MN,included,included


## Add the data to the study and save it

In [23]:
study.supplemental.waypoints = waypoints
study.supplemental.voyages = voyages_modalities

In [24]:
study.save('singlecell_pnm_figure4_voyages', flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')

Wrote datapackage to /projects/ps-yeolab/obotvinnik/flotilla_projects//singlecell_pnm_figure4_voyages/datapackage.json
